# Text Generation with Python and Keras
    Goal :
    - Process Text
    - Clean Text
    - Tokenize the Text and create Sequences with Keras

In [1]:
def read_file(filepath):
    with open(filepath) as f:
      str_text = f.read()
        
    return str_text

In [9]:
# read_file('moby_dick_four_chapters.txt')

In [15]:
import spacy

In [19]:
nlp = spacy.load('en_core_web_sm',disable=['parser','tagger','ner'])

In [21]:
nlp.max_length = 1198623

In [33]:
def separate_punc(doc_text):
    exclude = {'#', '$', '\n', '.', ',', '!', '?', ':', ';', '(', ')', '[', ']', '{', '}', '"', "'", '-', '_', '/', '\\', '|', '@', '%', '^', '&', '*', '~', '`', '+'}
    return [token.text.lower() for token in nlp(doc_text) if token.text not in exclude]

In [35]:
d = read_file('moby_dick_four_chapters.txt')

In [37]:
tokens = separate_punc(d)

In [38]:
len(tokens)

11851

In [42]:
train_len = 25 + 1
text_sequences = []
for i in range(train_len,len(tokens)):
    seq= tokens[i-train_len:i]
    text_sequences.append(seq)

In [44]:
type(text_sequences)

list

In [50]:
# text_sequences[1]
' '.join(text_sequences[0])

'call me ishmael   some years ago -- never mind how long precisely -- having little or no money in my purse and nothing particular to'

In [58]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [60]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)

In [62]:
sequences = tokenizer.texts_to_sequences(text_sequences)

In [70]:
# sequences[1]

In [68]:
tokenizer.index_word

{1: 'the',
 2: 'a',
 3: 'and',
 4: ' ',
 5: 'of',
 6: 'i',
 7: 'to',
 8: 'in',
 9: 'it',
 10: 'that',
 11: 'he',
 12: 'his',
 13: 'was',
 14: '\n\n',
 15: 'but',
 16: 'me',
 17: 'with',
 18: 'as',
 19: '--',
 20: 'at',
 21: 'this',
 22: 'you',
 23: 'is',
 24: 'all',
 25: 'for',
 26: 'my',
 27: 'on',
 28: 'be',
 29: "'s",
 30: 'not',
 31: 'from',
 32: 'there',
 33: 'one',
 34: 'up',
 35: 'what',
 36: 'him',
 37: 'so',
 38: 'bed',
 39: 'now',
 40: 'about',
 41: 'no',
 42: 'into',
 43: 'by',
 44: 'were',
 45: 'out',
 46: 'or',
 47: 'harpooneer',
 48: 'had',
 49: 'then',
 50: 'have',
 51: 'an',
 52: 'upon',
 53: 'little',
 54: 'some',
 55: 'old',
 56: 'like',
 57: 'if',
 58: 'they',
 59: 'would',
 60: 'do',
 61: 'over',
 62: 'landlord',
 63: 'thought',
 64: 'room',
 65: 'when',
 66: 'could',
 67: "n't",
 68: 'night',
 69: 'here',
 70: 'head',
 71: 'such',
 72: 'which',
 73: 'man',
 74: 'did',
 75: 'sea',
 76: 'time',
 77: 'other',
 78: 'very',
 79: 'go',
 80: 'these',
 81: 'more',
 82: 'th

In [74]:
 # tokenizer.word_counts

In [76]:
vocabulary_size = len(tokenizer.word_counts)

In [78]:
vocabulary_size

2721

In [80]:
import numpy as np

In [82]:
sequences = np.array(sequences)

In [84]:
sequences

array([[ 959,   16,  265, ...,  152,  261,    7],
       [  16,  265,    4, ...,  261,    7,  960],
       [ 265,    4,   54, ...,    7,  960,   16],
       ...,
       [ 955,   13,  168, ...,  264,   56,    2],
       [  13,  168, 2716, ...,   56,    2, 2721],
       [ 168, 2716,    3, ...,    2, 2721,   29]])